# Langflow에게 질문하기

In [ ]:
import requests
import time

def get_langflow_response(flow_id, idx, question):
    LANGFLOW_RUN_URL = f"http://localhost:7860/api/v1/run/{flow_id}" # Langflow 접근 URL
    print(f"[{idx}번째 질문]: {question}")  

    # API Key 인증이 비활성화되었으므로, 헤더에서 Authorization 및 Host 제거
    headers = {"Content-Type": "application/json"}
    
    # 안정적인 Payloads 구조 사용
    payload = {
        "output_type": "chat",
        "input_type": "chat",
        "input_value": question,
    }
    
    payload['session_id'] = f"{idx}번째 질문"

    response = requests.post(LANGFLOW_RUN_URL, json=payload, headers=headers)
    
    # HTTP 상태 코드 확인 (200 OK인지 확인)
    response.raise_for_status()
    
    flow_output = response.json()
    
    print("✅ 답변 완료!")
    
    return flow_output

In [ ]:
# Langflow ID
v1 = "fb1e9771-37e9-4ca4-932c-71425a651893" # Base (벡터 유사도만 활용)
v2 = "f192f1c5-da86-45d1-aacf-bef208cff621" # Multiquery (벡터 유사도만 활용)
v3 = "62f7afca-d898-4b51-83c2-a0bced1787f2" # Reranker (벡터 유사도만 활용)
v4 = "cc832bfc-65c2-4405-9f44-40eea9f6ab62" # MultiQuery
v5 = "49780fbb-07ee-4a46-b09a-0727a6c42b1c" # MultiQuery + Reranker

In [ ]:
# 질문 리스트
questions = [
    "납품검사 부실 및 직접생산 의무 위반으로 규격미달 자재가 반입된 사례와 유사 사례를 제시해주세요.",
    "중요 설계 기준(예: 중량 기준) 위반 사실을 인지하고도 시정 조치 없이 납품을 받은 경우, 관련자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "부당 병가 사용 및 연차수당 부당 수령 등 복무 관리 미흡 사례와, 이에 대한 징계 및 제도 개선이 이루어진 유사 사례를 제시해주세요.",
    "설계 기준 미준수 및 구조검토 미흡으로 사고가 발생했으나 관련자에게 '주의' 처분이 내려진 사례를 제시해 주세요.",
    "노무비 대리수령 관련 서류 미비 또는 부적정 처리로 '주의' 처분이 내려진 유사 사례를 제시해 주세요.",
    "정밀안전점검 결과를 부실하게 검토하여 안전등급을 실제보다 상향 또는 하향 지정한 사례를 제시해 주세요.",
    "지하안전관리계획 미이행 등으로 관리 소홀이 발생했으나 '주의' 처분에 그친 유사 사례를 제시해 주세요.",
    "사업비 대폭 증가 시 타당성 재조사 미실시 및 부당한 업무합의서 체결을 한 경우 어떤 처분을 내려야 하나요?",
    "공공공사에서 설계도면 미준수 및 관리감독 소홀로 부실 시공이 발생한 경우 위반되는 관련 법령과 조항을 알려주세요",
    "공공공사 하도급 현장에 자격 미달자를 배치하고 관리감독을 소홀히 한 경우 위반되는 법령과 조항을 제시해 주세요",
    "공공 건설공사 사후평가 미실시 및 부실 평가의 경우 어떤 처분을 내려야 할까?",
    "유해화학물질 취급시설에 유해화학물질의 명칭 및 위험 문구를 표시하지 않은 경우, 위반되는 관련 법령 또는 지침의 조항을 제시해 주세요.",
    "화약류 운반 시 경계요원을 탑승시키지 않는 등 운반방법을 준수하지 않은 경우, 위반되는 관련 법령이나 지침의 조항을 제시해 주세요.",
    "고압가스 사업장에서 안전관리책임자를 선임하지 않거나 안전관리자 전문교육을 이수하지 않은 경우, 위반되는 법령 또는 지침의 조항을 알려주세요.",
    "포상금이나 각종 수당을 상급자가 하급자에게 상납 요구하거나 사적으로 유용한 유사 사례를 찾아 제시해 주세요.",
    "내부지침 미준수 또는 경영공시 누락 등으로 '통보' 처분을 받은 사례를 2건 이상 제시해 주세요.",
    "기관 창립일이나 노조 창립일을 유급휴일로 운영하여 추가 수당을 지급한 경우, 위반되는 관련 규정과 조항을 제시해 주세요.",
    "성과급 부정 지급 및 환수 미이행과 관련하여 위반되는 법령이나 내부 규정의 조항을 제시해 주세요.",
    "계약보증금 미확보로 인해 계약 해지 또는 미지급금이 발생했던 유사 사례들을 제시해 주세요.",
    "소방설비 약제의 주기적 검사 미실시가 발생한 경우, 위반되는 관련 법령 또는 안전관리 규정의 조항을 제시해 주세요.",
    "국유지 무단점유에 대한 관리 소홀 및 변상금 부과 미이행에 따른 처분 수준을 추천해 주세요",
    "2020년 이후에 승진인원 임의 가산 등 인사운영의 공정성 훼손에 관한 사례를 제시해 주세요.",
    "채권 연체관리 소홀 및 압류 미이행과 관련하여 위반되는 법령 또는 내규의 조항을 제시해 주세요.",
    "자산의 과대계상 및 부적정 회계처리가 발생한 경우, 담당자 또는 기관에 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "사용자부담금 등 자산 및 비용 계정의 부적정 회계처리가 발생한 사례를 제시해 주세요.",
    "재정 전망 결과를 고의적으로 왜곡하거나 조작하는 행위는 어떤 법령 또는 내부 규정에 위반되는지 조항과 함께 제시해 주세요.",
    "신규 사업임에도 필수 심사 절차를 거치지 않고 예산을 편성한 유사 사례가 있었는지 찾아 제시해 주세요.",
    "근무하지 않은 인원에게 허위로 인건비를 지급하여 보조금을 횡령한 사례가 있었는지 찾아 제시해 주세요.",
    "특정 사업자에게 선정 특혜를 제공한 사례가 있었는지 찾아 제시해 주세요.",
    "수요조사나 타당성 검토 없이 공공사업을 추진하여 예산이 비효율적으로 집행된 유사 사례를 제시해 주세요",
    "사업 간 구간이나 일정 협의 없이 추진하여 도로 재시공 등 예산 낭비가 발생한 유사 사례를 제시해 주세요.",
    "건축허가 담당자의 신고서 검토 소홀로 무단 증축이 적법 승인된 유사 사례를 제시해 주세요.",
    "안전취약시설에 대해 긴급안전조치 없이 거주를 방치한 경우, 위반되는 관련 법령이나 규정의 조항을 제시해 주세요.",
    "중대결함 발생 시 이를 시스템에 누락 등록하거나 보수 조치를 미이행한 경우, 위반되는 법령이나 내규의 조항을 알려주세요.",
    "공공요금이나 부담금을 과다 산정하거나 이중 부과한 경우, 위반되는 관련 법령이나 기준의 조항을 제시해 주세요.",
    "공공공사 준공검사 시 부실시공을 확인하고도 시정조치 없이 준공 처리한 경우, 위반되는 관련 법령이나 기준의 조항을 제시해 주세요.",
    "보증 가입 요건 미흡으로 인해 고가주택에 대한 부적정한 보증 발급이 이루어진 경우, 담당자 또는 기관에 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "채권 회수 활동이 미흡하여 리스크 관리가 제대로 이루어지지 않은 경우, 담당자 또는 기관에 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "직무관련 업체로부터 금품을 수수하거나 부당한 계약을 체결하여 파면 또는 중징계 처분을 받았던 사례를 제시해 주세요.",
    "설계도서와 다르게 주요 구조 부재를 누락하거나 부적정하게 시공한 경우, 관련자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "공직자가 보유 현금재산 또는 재산 변동사항을 신고하지 않은 경우, 위반되는 법령과 해당 조항을 제시해 주세요.",
    "등기부등본 등 확인 소홀로 허위 임대차계약에 따른 부정대출이 발생한 유사 사례가 있다면 제시해 주세요.",
    "담보가치를 초과한 대출에 대해 인수 의무를 이행하지 않아 손실이 발생한 경우, 위반되는 관련 법령 또는 내규의 조항을 제시해 주세요.",
    "투자사업 추진 시 리스크 검토 미흡 및 부실한 협약 체결로 인한 정책자금 손실이 발생한 사례를 제시해 주세요",
    "업무용 자산의 사적 이용, 행사비 부당 집행과 관련된 유사 사례를 3건을 처분 수준이 높았던 사례 순서로 제시해 주세요.",
    "임직원 자녀가 거래업체에 취업하여 직무 관련 이익을 수수한 경우 어떤 처분을 내려야해?",
    "감리업무 수행 시 적법절차를 준수하지 않은 사례에 대해 알려줘.",
    "퇴직금 또는 상여금 산정기준이 불합리하여 과다 지급되어 주의 또는 통보 처분을 받은 사례를 제시해 주세요.",
    "파견자들이 복무규정을 위반하거나 업무실적이 부실하여 통보 또는 시정조치가 이루어진 유사 사례를 제시해 주세요.",
    "근무시간을 허위로 기재하여 수당을 부정하게 수령한 직원에 대해 어떤 처분 수준이 적절한지 추천해 주세요.",
    "법적 근거 없이 조직을 설치·운영하고 외부 인력을 과다하게 파견받아 운영한 경우 어떤 처분이 적절한지 추천해 주세요.",
    "내부 규정 위반이나 부정수령이 적발되었음에도 불구하고 주의 또는 경고 처분만 내려진 사례가 있다면 제시해 주세요.",
    "노동조합과의 임금협약 체결 및 예산 잔여금의 부적절한 전용과 관련하여 위반되는 법령 또는 내규의 조항을 구체적으로 제시해 주세요.",
    "경영실적보고서 작성 시 근로소득 항목 누락 및 허위 작성한 경우 어떤 처분을 내려야해?",
    "재해율 산정 기준의 불합리로 인해 사업장에 부당한 불이익이 발생한 경우, 담당자나 기관에 대해 어떤 처분 수준이 적절한지 추천해 주세요.",
    "감독 대상 선정 및 유해요인조사 미실시 등 안전보건조치 점검 소홀에 대해 적절한 처분 수준은 무엇인지 추천해 주세요.",
    "무면허 운전 여부를 제대로 확인하지 않고 산재처리 업무를 수행한 경우 관련 담당자나 기관에 대해 어떤 처분 수준이 적절한지 추천해 주세요.",
    "외국인 자격 관리 부실로 인해 취업이 불가능한 외국인에게 취업알선이 이루어진 경우 어떤 처분을 내려야해?",
    "개인정보 보유기간을 설정하지 않거나 장기간 과다하게 보유하는 행위는 어떤 법령 또는 지침의 어떤 조항을 위반하는지 제시해 주세요",
    "허위로 구직신청 및 취업 처리를 하여 실적을 부풀린 경우 어떤 처분을 내려야해?",
    "시험장비 교정주기 미설정 및 매뉴얼 미준수 등 교정 관리 부실 사례에 대해 알려주세요",
    "복리후생비 예산을 특정 명목으로 현금 지급한 행위가 위반하는 관련 예산집행지침의 조항을 제시해 주세요.",
    "보조금 지원 사업장 폐업 사실 미파악으로 보조금 환수 지연 및 미환수 사례가 다수 발생한 경우 어떤 처분을 내려야해?",
    "세무조사를 소홀히 하여 거래로 인한 증여세 부과가 누락된 경우, 담당자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "조세특례 요건을 충족하지 않은 법인에 세액공제나 감면을 과다 적용하여 법인세가 과소 신고된 경우, 담당자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "탈세제보 처리 시 제출된 허위 증빙을 면밀히 검토하지 않고 부당하게 처리한 경우, 담당자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "비과세 또는 감면 대상에 대해 잘못 세금을 부과하여 환급 또는 시정조치가 이루어진 사례를 모두 제시해 주세요",
    "납세의무자를 잘못 지정하여 납세자 권익이 침해된 사례를 모두 제시해 주세요.",
    "승인 절차 없이 세무조사 기간 연장 또는 범위 확대 등 조사권을 남용한 사례를 모두 제시해 주세요.",
    "공동주택 등 자산 평가 시 평가심의 절차를 거치지 않아 과세가액이 과소 산정된 유사 사례를 제시해 주세요.",
    "감정평가심의회 운영이 미흡하여 자산 평가의 공정성이 저해된 사례를 제시해 주세요",
    "양도차익이 과소 산정되어 양도소득세가 부족 징수된 유사 사례를 제시해 주세요.",
    "상속세 조사 과정에서 부동산 감정평가 선정 및 철회 절차를 부당하게 처리하여 부족 징수가 발생한 경우 어떤 처분을 내려야해?",
    "비상장주식 평가 기준 미준수로 인한 상속세 부족 징수 사례를 제시해 주세요",
    "공동소유 토지 및 건물부속토지 평가 기준 미준수로 인한 비상장주식 과소평가와 관련하여 위반되는 법령 또는 내부 지침의 조항을 제시해 주세요.",
    "2020년 이전에 보상금 지급업무 처리 과정에서 행정절차를 위반한 사례를 찾아주고, 해당 기관명도 함께 알려주세요.",
    "사업비 증액 후 심사 미실시 및 인건비 중복 수령 등 부적정 집행이 발생한 경우와 유사한 사례를 2건 제시해 주고, 각각 내용과 처분을 요약해주세요",
    "채용 과정에서 자격요건을 충족하지 않은 지원자를 채용한 경우, 위반되는 인사규정 또는 채용지침의 조항을 제시해 주세요.",
    "무허가 업체와의 계약으로 담당자가 징계 처분을 받았던 사례를 제시해 주세요.",
    "구체적 사업계획 없이 용도지역을 부당하게 변경하거나 불필요한 토지 매입을 한 사례를 제시해 주세요",
    "계약 체결 과정에서 허위 보고서 작성, 내부문서 유출, 입찰 절차 미준수 등 복합적인 위반이 발생한 사례를 제시하고,각각 적절한 처분 수준을 추천해 주세요",
    "행정 결정 과정에서 사실과 다른 내용을 근거로 절차를 강행하거나 민원에 부당하게 대응한 경우, 관련자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "산업단지에 입주자격을 갖추지 않은 시설이 부당하게 입주하거나, 관리 감독이 미흡하여 적절한 조치가 이루어지지 않은 유사 사례를 찾아 제시해 주세요.",
    "민간위탁사업 등 선정업무 절차 위반으로 '주의' 처분을 받았던 사례를 3건 이상 제시해 주세요",
    "근무지 무단이탈과 시간외근무수당 부당 수령이 동시에 발생한 경우, 담당자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "병가 또는 공가를 목적에 맞지 않게 사용하거나 무단으로 국외여행을 한 경우, 담당자에게 내릴 수 있는 적절한 처분 수준은 무엇입니까?",
    "공공사업 추진 과정에서 민간사업자에게 재정적 특혜를 제공한 사례를 제시해 주세요",
    "공공사업에서 투자심사 승인 전 사업을 착수하거나 심사 통과를 위해 사업비를 축소 제출한 경우 어떤 처분을 내려야 할지 추천해 주세요",
    "공공사업에서 중요재산의 처분을 승인 권한 없이 허가한 경우 어떤 처분을 내려야해?",
    "부동산 취득 후 취득세를 신고·납부하지 않은 경우, 위반되는 관련 세법 및 조항을 알려주세요.",
    "법령 개정안 검토 업무를 소홀히 하여 위법 행위가 발생한 경우, 위반되는 관련 법령 및 조항을 알려주세요.",
    "부담금 부과 대상이 아닌 항목을 포함하여 과다 부과가 발생한 경우, 담당자에게 적절한 처분 수준은 무엇인가?",
    "부담금 산정 시 면적 계산 오류로 인해 미부과가 발생한 사례에 대해 알려주세요",
    "뭐에 대해 답변해줄 수 있어?",
    "어떤 감사사례에 대해 알고있어?",
    "오늘 날씨에 대해 알려줘",
    "넌 이런 질문에서 처분 수위는 어떤 기준으로 판단해?",
    "실제 사례가 없으면 답변을 못 하는지, 가상 사례도 가능한지 알려줘.",
]

In [ ]:
for idx, q in enumerate(questions, 1):
    output = get_langflow_response(v4, idx, q)
    time.sleep(1)

In [ ]:
for idx, q in enumerate(questions, 1):
    output = get_langflow_response(v5, idx, q)
    time.sleep(1)